In [1]:
import openai

with open("../openai_api_key.txt", "r") as f:
    api_key = f.read()

# openai.api_key = api_key

GPT_model = "gpt-3.5-turbo"

# ENGINE = "code-davinci-002"
# MAX_TOKENS = 512
# N_SAMPLES = 1
# TEMPERATURE = 0.0
# STOP = "/**"

In [2]:
RANDOM_SEED = 123

client = openai.Client(api_key=api_key)

In [3]:
# read descriptions from json file
from data_load import load_games, load_system_prompt

games = load_games()
system_prompt = load_system_prompt()

In [4]:
games

[Name: A Simple Game
 Description: This game is played on the following 4x4 square board. MOVE - On each turn, each player move one of his stones into an adjacent orthogonal empty cell. GOAL - Wins the player that makes a (orthogonal or diagonal) 3 in-a-row.
 Code: (game "A Simple Game"
     (players 2)
     (equipment {
         (board (square 4) use:Vertex)
         (piece "Ball" Each ("StepToEmpty" Orthogonal))
         (hand Each)
     })
     (rules
         (start {
             (place "Ball1" (intersection (sites Phase 0) (union (sites Top) (sites Bottom))))
             (place "Ball2" (intersection (sites Phase 1) (union (sites Top) (sites Bottom))))
         })
         (play (forEach Piece))
         (end ("Line3Win" All))
     )
 ),
 Name: 3D Tic Tac Toe
 Description: Play occurs on a 3x3x3 grid. One player places an X, the other places an O and players take turns placing their marks in the grid, attempting to get three in a row of their color. For the lines between levels, 

In [6]:
def get_model_response(message):
    # return ""
    completion = client.chat.completions.create(
    model=GPT_model,
    seed=RANDOM_SEED,
    messages=message,
    )
    return completion.choices[0].message.content

In [7]:
def get_naive_message(game):
    messages = []
    messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": game.description})
    return messages


In [8]:
naive_model_response = get_model_response(get_naive_message(games[0]))

In [9]:
print(naive_model_response)

boardheight = 4
boardwidth = 4

piececolors = ["red", "blue"]

piecetypes = ["stone"]

winconditions = [
    "3inarow"
]

initialPosition = [
    "........",
    "........",
    "........",
    "........"
]


In [10]:
def get_message_with_example(game_question, game_examples):
    messages = []
    
    messages.append({"role": "system", "content": system_prompt})
    for example in game_examples:
        messages.append({"role": "user", "content": example.description})
        messages.append({"role": "assistant", "content": example.code})
    messages.append({"role": "user", "content": game_question.description})
    return messages

In [14]:
better_model_response = get_model_response(get_message_with_example(games[0], games[1:4]))

In [17]:
games

[Name: A Simple Game
 Description: This game is played on the following 4x4 square board. MOVE - On each turn, each player move one of his stones into an adjacent orthogonal empty cell. GOAL - Wins the player that makes a (orthogonal or diagonal) 3 in-a-row.
 Code: (game "A Simple Game"
     (players 2)
     (equipment {
         (board (square 4) use:Vertex)
         (piece "Ball" Each ("StepToEmpty" Orthogonal))
         (hand Each)
     })
     (rules
         (start {
             (place "Ball1" (intersection (sites Phase 0) (union (sites Top) (sites Bottom))))
             (place "Ball2" (intersection (sites Phase 1) (union (sites Top) (sites Bottom))))
         })
         (play (forEach Piece))
         (end ("Line3Win" All))
     )
 ),
 Name: 3D Tic Tac Toe
 Description: Play occurs on a 3x3x3 grid. One player places an X, the other places an O and players take turns placing their marks in the grid, attempting to get three in a row of their color. For the lines between levels, 

In [15]:
games[1]

Name: 3D Tic Tac Toe
Description: Play occurs on a 3x3x3 grid. One player places an X, the other places an O and players take turns placing their marks in the grid, attempting to get three in a row of their color. For the lines between levels, the diagonals are not taken in account.
Code: (define "Line3D" 
    (!= 0 
        (count Sites
            in:(forEach 
                (expand (sites Bottom) steps:(- 3 1))
                if:(all Sites (sites {(site) (ahead (site) steps:4 N) (ahead (site) steps:8 N)})
                    if:(= (mover) (who at:(site)))
                )
            )
        )
    )
)

//------------------------------------------------------------------------------

(game "3D Tic-Tac-Toe"
    (players 2)
    (equipment {
        (board (rectangle (+ (^ 3 2) (- 3 1)) 3))
        (piece "Disc" P1)
        (piece "Cross" P2)
        (piece "Dot" Neutral)
    })
    (rules
        (start (place "Dot0" (union {(sites Row 3) (sites Row 7)}))) // Places neutral dots t

In [16]:
print(better_model_response)

(game "4x4 Tic-Tac-Toe"
    (players 2)
    (equipment {
        (board (square 4))
        (piece "X" P1)
        (piece "O" P2)
    })
    (rules
        (play (move Move (from (sites Occupied by:Mover)) (to (sites Empty Orthogonal))))
        (end (if (is Line 3) (result Mover Win)))
    )
)
